In [2]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install 'accelerate>=0.26.0'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
# from transformers import pipeline

# # Load a pretrained sentiment analysis model
# sentiment_analysis = pipeline("sentiment-analysis")

# # Example review text from a manager
# manager_review = "John has been showing a lot of improvement in his leadership skills."

# # Get the sentiment of the review
# result = sentiment_analysis(manager_review)
# print(result)



import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

# Define a prompt for your POC
prompt = "Evaluate the employee performance based on the following review:"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

# Generate the output
output = model.generate(**inputs, max_length=200,temperature=0.7,top_k=50, num_return_sequences=1)

# Decode the response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


/home/shams/Desktop/genAI/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  9.77it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


KeyboardInterrupt: 

In [2]:
structured_data = {
    "attendance": 92,        # Attendance percentage
    "courses_completed": 6,  # Number of training courses completed
    "manager_rating": 4.2    # Manager’s rating out of 5
}


In [3]:
review_text = "John has been punctual but needs to improve his technical skills."


In [4]:
prompt = (
    f"Employee performance summary:\n"
    f"Attendance: {structured_data['attendance']}%\n"
    f"Training Completed: {structured_data['courses_completed']} courses\n"
    f"Manager Rating: {structured_data['manager_rating']}/5\n"
    f"Manager Review: {review_text}\n\n"
    f"Generate a detailed performance summary:"
)

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

# Generate the response from the LLaMA model
output = model.generate(**inputs, max_length=200)

# Decode the output
performance_summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(performance_summary)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Employee performance summary:
Attendance: 92%
Training Completed: 6 courses
Manager Rating: 4.2/5
Manager Review: John has been punctual but needs to improve his technical skills.

Generate a detailed performance summary: Employee performance summary: Attendance: 92% Training Completed: 6 courses Manager Rating: 4.2/5 Manager Review: John has been punctual but needs to improve his technical skills.



In [6]:
def classify_performance(attendance, courses, rating):
    score = (attendance * 0.4) + (courses * 5) + (rating * 10)
    print(score)
    if score >= 85:
        return "Excellent"
    elif 70 <= score < 85:
        return "Very Good"
    elif 55 <= score < 70:
        return "Average"
    elif 35 <= score < 55:
        return "Below Average"
    else:
        return "Poor"

performance_level = classify_performance(
    structured_data["attendance"],
    structured_data["courses_completed"],
    structured_data["manager_rating"]
)
print(f"Performance Level: {performance_level}")


108.80000000000001
Performance Level: Excellent


#working on managerial review

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

# Load the Meta-Llama-3-8B model and tokenizer
model_name = "meta-llama/Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Sample DataFrame with review text and manager ratings
data = pd.DataFrame({
    'review_text': [
        "Shams is very punctual and arrives on time every day.",
        "Needs improvement in technical skills but good punctuality.",
        "Highly skilled and punctual, delivering tasks on time.",
        "Frequently late, needs better technical knowledge."
    ],
    'manager_rating': [8, 6, 9, 5]
})

# Function to analyze review text using the Llama model
def analyze_review(text):
    # Prepare input prompt for the model
    prompt = f"Evaluate the following review on Punctuality and Technical Skills out of 10:\n\n{text}\n\nResponse:"
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract scores from the model's response
    try:
        punctuality = int(result.split("Punctuality: ")[1].split()[0])
        technical_skills = int(result.split("Technical Skills: ")[1].split()[0])
    except:
        punctuality, technical_skills = 7, 7  # Default score if parsing fails

    return punctuality, technical_skills

# Apply the model analysis to each review
data[['Punctuality_Score', 'Technical_Skills_Score']] = data['review_text'].apply(
    lambda x: pd.Series(analyze_review(x))
)

# Normalize Manager Rating to 10 if needed (assuming already out of 10 here)
data['Manager_Rating'] = data['manager_rating']

# Calculate Overall Score and Status
data['Overall_Score'] = data[['Punctuality_Score', 'Technical_Skills_Score', 'Manager_Rating']].mean(axis=1)

def assign_status(score):
    if score >= 8:
        return "Excellent"
    elif score >= 6:
        return "Good"
    else:
        return "Needs Improvement"

data['Overall_Status'] = data['Overall_Score'].apply(assign_status)

# Display the final DataFrame
print(data[['review_text', 'Punctuality_Score', 'Technical_Skills_Score', 
            'Manager_Rating', 'Overall_Score', 'Overall_Status']])


/home/shams/Desktop/genAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: meta-llama/Llama-3-8B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`